In [4]:
import os
import glob
import rasterio
from rasterio.merge import merge
from rasterio.plot import show

OUTPUT_DIR = ""
MOSAIC_DIR = ""
os.makedirs(MOSAIC_DIR, exist_ok=True)

def mosaic_rasters(pattern, output_path):
    # Get all matching raster files
    raster_files = glob.glob(pattern)
    if not raster_files:
        print(f"⚠️ No rasters found for pattern: {pattern}")
        return

    src_files_to_mosaic = [rasterio.open(fp) for fp in raster_files]

    # Merge rasters
    mosaic, out_trans = merge(src_files_to_mosaic)

    # Copy metadata from one of the input files and update
    out_meta = src_files_to_mosaic[0].meta.copy()
    out_meta.update({
        "driver": "GTiff",
        "height": mosaic.shape[1],
        "width": mosaic.shape[2],
        "transform": out_trans
    })

    # Write the mosaic raster
    with rasterio.open(output_path, "w", **out_meta) as dest:
        dest.write(mosaic)

    # Close all raster files
    for src in src_files_to_mosaic:
        src.close()

    print(f"✅ Mosaic saved to: {output_path}")

def main():
    # Define file patterns and output file names
    raster_types = ["dsm", "dtm", "ndsm", "intensity"]

    for raster_type in raster_types:
        pattern = os.path.join(OUTPUT_DIR, f"*_{raster_type}.tif")
        output_path = os.path.join(MOSAIC_DIR, f"{raster_type}_mosaic.tif")
        print(f"🧩 Mosaicking {raster_type.upper()} rasters...")
        mosaic_rasters(pattern, output_path)

    print("🎉 All mosaics created!")

In [5]:
main()

🧩 Mosaicking DSM rasters...
✅ Mosaic saved to: /Users/main/grad_school/term2_assessments/GeoAI/lidar_mosaic/dsm_mosaic.tif
🧩 Mosaicking DTM rasters...
✅ Mosaic saved to: /Users/main/grad_school/term2_assessments/GeoAI/lidar_mosaic/dtm_mosaic.tif
🧩 Mosaicking NDSM rasters...
✅ Mosaic saved to: /Users/main/grad_school/term2_assessments/GeoAI/lidar_mosaic/ndsm_mosaic.tif
🧩 Mosaicking INTENSITY rasters...
✅ Mosaic saved to: /Users/main/grad_school/term2_assessments/GeoAI/lidar_mosaic/intensity_mosaic.tif
🎉 All mosaics created!
